# Team Stacks Mart Preview

Preview the team stacks analysis mart outputs for NFL DK Classic.

In [5]:
import os
import duckdb

# S3 Configuration
s3_endpoint = os.getenv("WASABI_ENDPOINT", "s3.us-east-2.wasabisys.com")
s3_access_key = os.getenv("WASABI_ACCESS_KEY")
s3_secret_key = os.getenv("WASABI_SECRET_KEY")
bucket_name = os.getenv("WASABI_BUCKET_NAME")

# Initialize DuckDB connection
con = duckdb.connect()

# Configure S3
con.execute(f"""
    SET s3_endpoint='{s3_endpoint}';
    SET s3_access_key_id='{s3_access_key}';
    SET s3_secret_access_key='{s3_secret_key}';
    SET s3_url_style='path';
""")

# Base path for team stacks mart
base_path = f"s3://{bucket_name}/marts/team_stacks/NFL/dk_classic"

print(f"✓ Connected to DuckDB")
print(f"✓ Base path: {base_path}")

✓ Connected to DuckDB
✓ Base path: s3://dfscrunch-data-lake/marts/team_stacks/NFL/dk_classic


## 1. Stack Sizes Distribution

Overall distribution of stack size combinations across all lineups.

In [6]:
# Read stack_sizes table
stack_sizes = con.execute(f"""
    SELECT *
    FROM read_parquet('{base_path}/stack_sizes.parquet')
    ORDER BY lineup_count DESC
    LIMIT 15
""").df()

print("\nTop 15 stack combinations:")
stack_sizes


Top 15 stack combinations:


,stack_combination,lineup_count,pct_of_lineups,avg_total_own,avg_lineup_rank
0,2,4939,73.11,145.43,18.1
1,3,3147,46.58,147.90,17.5
2,"3,2",1603,23.73,134.32,17.7
3,"2,2",1568,23.21,128.40,19.6
4,4,778,11.52,204.71,14.1
5,"2,2,2",388,5.74,143.64,17.3
6,"3,2,2",368,5.45,170.00,14.6
7,"4,2",354,5.24,189.89,14.7
8,5,262,3.88,229.88,12.6
9,"3,3",229,3.39,212.98,12.2


## 2. Team Stack Details

Per-team breakdown with position combinations and performance metrics.

In [7]:
# Read team_details table
team_details = con.execute(f"""
    SELECT *
    FROM read_parquet('{base_path}/team_details.parquet')
    ORDER BY lineup_count DESC
    LIMIT 20
""").df()

# print(f"Total rows: {con.execute(f'SELECT COUNT(*) FROM read_parquet(\'{base_path}/team_details.parquet\')').fetchone()[0]}")
print("\nTop 20 team stacks by frequency:")
team_details


Top 20 team stacks by frequency:


,team,stack_size,lineup_count,pct_of_lineups,position_combo,position_combo_count,avg_total_own,avg_lineup_rank
0,SF,2,559,8.27,"QB,WR",227,163.14,17.9
1,HOU,2,440,6.51,"RB,WR",228,135.42,20.4
2,TB,2,402,5.95,"RB,WR",158,133.94,18.0
3,IND,2,316,4.68,"RB,WR",109,137.77,17.9
4,ATL,2,314,4.65,"RB,WR",165,143.20,17.3
5,LA,2,298,4.41,"RB,WR",143,148.09,17.8
6,GB,2,288,4.26,"DST,RB",87,174.78,16.1
7,DET,2,286,4.23,"RB,WR",89,158.56,16.4
8,BUF,2,280,4.14,"QB,WR",107,178.34,16.8
9,CHI,2,277,4.10,"QB,WR",115,149.95,18.2


## 3. QB Stacking Patterns

QB correlation analysis - how often QB is stacked with WR, TE, RB.

In [8]:
# Read qb_patterns table
qb_patterns = con.execute(f"""
    SELECT *
    FROM read_parquet('{base_path}/qb_patterns.parquet')
    ORDER BY lineup_count DESC
""").df()

print(f"Total QB stacking patterns:")
qb_patterns

Total QB stacking patterns:


,stack_type,lineup_count,pct_of_qb_stacks,avg_total_own,avg_lineup_rank
0,QB + WR (no TE),3870,61.75,142.18,18.3
1,QB + WR + TE,1743,27.81,157.94,16.8
2,QB not stacked,489,7.80,122.93,19.1
3,QB + TE (no WR),484,7.72,136.67,18.5
4,QB + RB (no pass catchers),170,2.71,155.45,17.7


## 4. DST Stacking Patterns

DST stacking analysis - frequency and position combinations.

In [9]:
# Read dst_patterns table
dst_patterns = con.execute(f"""
    SELECT *
    FROM read_parquet('{base_path}/dst_patterns.parquet')
    ORDER BY lineup_count DESC
""").df()

print(f"DST stacking patterns:")
dst_patterns

DST stacking patterns:


,dst_in_stack,lineup_count,pct_of_lineups,most_common_position_combo,combo_count,avg_total_own,avg_lineup_rank
0,False,3975,58.84,"DST,RB",1049,125.17,19.7
1,True,2781,41.16,"DST,RB",1049,172.83,15.6


## 5. Summary Statistics

In [10]:
# Get summary statistics
summary = {
    "Table": ["stack_sizes", "team_details", "qb_patterns", "dst_patterns"],
    "Row Count": [
        con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_path}/stack_sizes.parquet')").fetchone()[0],
        con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_path}/team_details.parquet')").fetchone()[0],
        con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_path}/qb_patterns.parquet')").fetchone()[0],
        con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_path}/dst_patterns.parquet')").fetchone()[0],
    ]
}

import pandas as pd
summary_df = pd.DataFrame(summary)
print("\nTeam Stacks Mart Summary:")
summary_df


Team Stacks Mart Summary:


,Table,Row Count
0,stack_sizes,27
1,team_details,127
2,qb_patterns,5
3,dst_patterns,2


## Cleanup

In [ ]:
# Close connection
con.close()
print("✓ Connection closed")